In [1]:
import os
import warnings
import random
from os.path import join
import torch
import numpy as np
import pandas as pd
from scipy import sparse
from collections import OrderedDict

In [2]:
torch.__version__

'1.9.0'

In [3]:

from utils import load_data_sequential
from utils import eval_sequential
import warnings
import random
from os.path import join
import torch
import numpy as np


warnings.filterwarnings('ignore')


def seed_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

seed = 1
seed_everything(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

top_k = 20

Using backend: pytorch


In [4]:
import json
import re
with open('../usrlog.txt', 'r') as f:
    data = f.read()
    data = data.replace("\'", "\"")

In [6]:
import ast

data = ast.literal_eval(data)
print(type(data))

<class 'dict'>


In [7]:
playlists = data.values()

In [8]:
playlists = [list(OrderedDict.fromkeys(l)) for l in playlists]

In [9]:
playlists = [i for i in playlists if len(i)>2]

In [10]:
len(playlists)

58730

In [11]:
song_set = set()
song_counter = dict()
for l in playlists:
    for song in l:
        song_set.add(song)
        song_counter[song] = song_counter.get(song, 0) + 1

In [12]:
len(song_set)

119061

In [13]:
freq_song_list = [k for (k, v) in list(song_counter.items()) if v > 5]

In [14]:
freq2id = dict()
for idx, i in enumerate(freq_song_list):
    freq2id[i] = idx

In [15]:
freq_playlists = [[s for s in l if s in freq_song_list] for l in playlists]

In [16]:
freq_playlists = [i for i in freq_playlists if len(i)>2]

In [17]:
freq_playlists = [[freq2id[s] for s in l] for l in freq_playlists]

In [18]:
len( freq_playlists)

46803

In [19]:
len(freq_playlists)

46803

In [20]:
count = 0
for i in freq_playlists:
    count+=len(i)

In [21]:
1 - count / len(freq_playlists) / 15984

0.9994851217262174

In [22]:
top_k = 500

In [23]:
num_users = len(freq_playlists)
num_items = len(freq_song_list)

print(f"# of users: {num_users},  # of items: {num_items}")

music_data = dict()
for idx, i in enumerate(freq_playlists):
    music_data[idx] = i
    
    
user_train_music = dict()
user_valid_music = dict()
user_test_music = dict()

for u in music_data:
    user_valid_music[u] = [music_data[u][-2]]
    user_test_music[u] = [music_data[u][-1]]
    user_train_music[u] = music_data[u][:-2]

# of users: 46803,  # of items: 15984


In [24]:
top_k = 500
top_k

500

In [25]:

from models.TransRec_sequential import TransRec_sequential

seed_everything(seed)
TransRec = TransRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, emb_dim=200, maxlen=1,
                                num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 500)
TransRec.fit()
TransRec_ndcg, TransRec_recall = eval_sequential(TransRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, 500, mode='test')
print(f"\n[TransRec]\t Test_Recall@{top_k} = {TransRec_recall:.4f} Test_NDCG@{top_k} = {TransRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in TransRec.parameters() if p.requires_grad))
print("======================================")

valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 410.88it/s]


epoch 1 train_loss = 0.59549397 valid_recall@500 = 0.3070 valid_ndcg@500 = 0.0591


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 402.03it/s]


epoch 2 train_loss = 0.50070167 valid_recall@500 = 0.2860 valid_ndcg@500 = 0.0511


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 399.06it/s]


...

epoch 5 train_loss = 0.15677644 valid_recall@500 = 0.3020 valid_ndcg@500 = 0.0556


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:02<00:00, 392.47it/s]


Early stop at epoch:6


test: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46803/46803 [02:07<00:00, 367.90it/s]


[TransRec]	 Test_Recall@500 = 0.2592 Test_NDCG@500 = 0.0472
parmas : 12573584


In [26]:
"""
SASRec
"""
from models.SASRec_sequential import SASRec_sequential

seed_everything(seed)
SASRec = SASRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, hidden_dim=50, maxlen=20, num_blocks=1, num_heads=1,
                            num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 500)
SASRec.fit()
SASRec_ndcg, SASRec_recall = eval_sequential(SASRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k, mode='test')
print(f"\n[SASRec]\t Test_Recall@{top_k} = {SASRec_recall:.4f} Test_NDCG@{top_k} = {SASRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in SASRec.parameters() if p.requires_grad))

valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 301.42it/s]


epoch 1 train_loss = 2.5220 valid_recall@500 = 0.0440 valid_ndcg@500 = 0.0056


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 301.87it/s]


epoch 2 train_loss = 1.5141 valid_recall@500 = 0.0460 valid_ndcg@500 = 0.0063


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 301.72it/s]


...

epoch 21 train_loss = 1.0769 valid_recall@500 = 0.2890 valid_ndcg@500 = 0.0559


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 302.88it/s]


Early stop at epoch:22


test: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46803/46803 [02:40<00:00, 291.74it/s]



[SASRec]	 Test_Recall@500 = 0.2620 Test_NDCG@500 = 0.0472
parmas : 815850


In [27]:
"""
BERT4Rec
"""
from models.BERTRec_sequential import BERTRec_sequential

seed_everything(seed)
BERTRec = BERTRec_sequential(user_train_music, user_valid_music, user_num=num_users, item_num=num_items, hidden=50*5, maxlen=50, n_layers=2, heads=5, mask_prob=0.1,
                                num_epochs=100, eval_every=1, early_stop_trial=5, learning_rate=0.001, reg_lambda=0.0, batch_size=128, device=device, top_k = 500)
BERTRec.fit()
BERTRec_ndcg, BERTRec_recall = eval_sequential(BERTRec, user_train_music, user_valid_music, user_test_music, num_users, num_items, top_k, mode='test')
print(f"\n[BERTRec]\t Test_Recall@{top_k} = {BERTRec_recall:.4f} Test_NDCG@{top_k} = {BERTRec_ndcg:.4f}")
print("parmas :", sum(p.numel() for p in BERTRec.parameters() if p.requires_grad))

valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 223.46it/s]


epoch 1 train_loss = 10.0465 valid_recall@500 = 0.2120 valid_ndcg@500 = 0.0385


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 223.59it/s]


epoch 2 train_loss = 9.4063 valid_recall@500 = 0.3080 valid_ndcg@500 = 0.0567


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 223.17it/s]


...

epoch 6 train_loss = 9.4751 valid_recall@500 = 0.2640 valid_ndcg@500 = 0.0512


valid: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 224.18it/s]


Early stop at epoch:7


test: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46803/46803 [03:16<00:00, 238.32it/s]


[BERTRec]	 Test_Recall@500 = 0.2463 Test_NDCG@500 = 0.0451
parmas : 9527735


In [28]:
train_rows = []
for user in user_train_music:
    for song in user_train_music[user]:
        train_rows.append((user,song,1))
train = pd.DataFrame(train_rows, columns = ['user_id', 'item_id', 'rating'])

In [29]:
valid_rows = []
for user in user_valid_music:
    for song in user_valid_music[user]:
        valid_rows.append((user,song,1))
valid = pd.DataFrame(valid_rows, columns = ['user_id', 'item_id', 'rating'])

In [30]:
test_rows = []
for user in user_test_music:
    for song in user_test_music[user]:
        test_rows.append((user,song,1))
test = pd.DataFrame(test_rows, columns = ['user_id', 'item_id', 'rating'])

In [31]:
train = train.to_numpy()
valid = valid.to_numpy()
test = test.to_numpy()

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in train:
    matrix[u, i] = r
train = sparse.csr_matrix(matrix)

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in valid:
    matrix[u, i] = r
valid = sparse.csr_matrix(matrix)

matrix = sparse.lil_matrix((num_users, num_items))
for (u, i, r) in test:
    matrix[u, i] = r
test = sparse.csr_matrix(matrix)

# train = train.toarray()
# valid = valid.toarray()
# test = test.toarray()

In [32]:
train = train+valid

In [33]:
top_k = 500

In [34]:
"""
Embarrassingly shallow autoencoders for sparse data, 
Harald Steck,
Arxiv.
"""
import os
import math
import numpy as np

class EASE_implicit():
    def __init__(self, train, valid, reg_lambda):
        self.train = train.toarray()
        self.valid = valid.toarray()
        self.num_users = train.shape[0]
        self.num_items = train.shape[1]
        self.reg_lambda = reg_lambda

    def fit(self):   
        train_matrix = self.train
        G = train_matrix.T @ train_matrix
        diag = np.diag_indices(G.shape[0]) 
        G[diag] += self.reg_lambda 
        P = np.linalg.inv(G)
        
        
        self.enc_w = P / (-np.diag(P)) 
        self.enc_w[diag] = 0

        self.reconstructed = self.train @ self.enc_w

    def predict(self, user_id, item_ids):
        return self.reconstructed[user_id, item_ids]

In [35]:
from utils import eval_implicit

In [36]:
"""
EASE
"""
# from models.EASE_implicit import EASE_implicit

seed_everything(seed)
ease = EASE_implicit(train=train, valid=valid, reg_lambda=100)
ease.fit()

In [37]:
ease_prec, ease_recall, ease_ndcg = eval_implicit(ease, train.toarray(), test.toarray(), 20)
print("EASE Prec@20: %f, Recall@20: %f, NDCG@20: %f"%(ease_prec, ease_recall, ease_ndcg))

EASE Prec@20: 0.004302, Recall@20: 0.086041, NDCG@20: 0.037489


In [38]:
ease_prec, ease_recall, ease_ndcg = eval_implicit(ease, train.toarray(), test.toarray(), 500)
print("EASE Prec@50: %f, Recall@50: %f, NDCG@50: %f"%(ease_prec, ease_recall, ease_ndcg))

EASE Prec@50: 0.000639, Recall@50: 0.319552, NDCG@50: 0.072306
